# 0. Goal
In this project we design a system that can accurately predict email recipients for a new email, given the user’s email history.

In [1]:
import numpy as np
import pandas as pd
import datetime
import pickle
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression

# 1. Load Data

In [2]:
training = pd.read_csv('./data/training_set.csv', sep=',', header=0)
training_info = pd.read_csv('./data/training_info.csv', sep=',', header=0, parse_dates=['date'])
test = pd.read_csv('./data/test_set.csv', sep=',', header=0)
test_info = pd.read_csv('./data/test_info.csv', sep=',', header=0, parse_dates=['date'])

In [3]:
print(training.shape)
training.head()

(125, 2)


,sender,mids
0,karen.buckley@enron.com,158713 158697 200301 158679 278595 298162 2002...
1,amr.ibrahim@enron.com,215241 3437 215640 3506 191790 3517 3520 3562 ...
2,andrea.ring@enron.com,270705 270706 270707 270708 270709 270710 2707...
3,sylvia.hu@enron.com,111444 111422 183084 111412 111347 110883 1105...
4,phillip.platter@enron.com,327074 327384 327385 264443 274124 274125 2741...


In [4]:
print(training_info.shape)
print(training_info.dtypes)
# date is not parsed: there must be an error on format
training_info.head()

(43613, 4)
mid            int64
date          object
body          object
recipients    object
dtype: object


,mid,date,body,recipients
0,60,2000-07-25 08:14:00,Legal has been assessing the risks of doing bl...,robert.badeer@enron.com murray.o neil@enron.co...
1,66,2000-08-03 02:56:00,Attached is a spreadsheet to estimate export f...,kim.ward@enron.com robert.badeer@enron.com mur...
2,74,2000-08-15 05:37:00,Kevin/Bob: Here is a quick rundown on the cons...,robert.badeer@enron.com john.massey@enron.com ...
3,80,2000-08-20 14:12:00,check this out and let everyone know what s up...,robert.badeer@enron.com jeff.richter@enron.com
4,83,2000-08-22 08:17:00,Further to your letter to us (addressed to Mr....,pgillman@schiffhardin.com kamarlantes@calpx.co...


In [5]:
print(test.shape)
test.head()

(125, 2)


,sender,mids
0,karen.buckley@enron.com,298389 332383 298390 284071 366982 81773 81791...
1,amr.ibrahim@enron.com,48260 48465 50344 48268 50330 48237 189979 189...
2,andrea.ring@enron.com,366364 271168 271172 271167 271189
3,sylvia.hu@enron.com,134931 134856 233549 233517 134895 233584 3736...
4,phillip.platter@enron.com,274220 274225 274215 274223 274214 274207 2742...


In [6]:
print(test_info.shape)
print(test_info.dtypes)
test_info.head()

(2362, 3)
mid              int64
date    datetime64[ns]
body            object
dtype: object


,mid,date,body
0,1577,2001-11-19 06:59:51,Note: Stocks of heating oil are very high for...
1,1750,2002-03-05 08:46:57,"Kevin Hyatt and I are going for ""sghetti"" at S..."
2,1916,2002-02-13 14:17:39,This was forwarded to me and it is funny. - Wi...
3,2094,2002-01-22 11:33:56,I will be in to and happy to assist too. I ma...
4,2205,2002-01-11 07:12:19,Thanks. I needed a morning chuckle.


# 2.Data processing 

## 2.1 Correct date format in training_info
Some dates are like 0001-08-26 22:16:36 (correct value should be 2001-08-26 22:16:36)

In [7]:
# Iterate over DataFrame rows as (index, Series) pairs and correct thme 
for row in training_info.sort_values(['date']).iterrows():
    date = row[1]['date']
    if date[:3] == '000':
        date = '2' + date[1:]      
    training_info.loc[row[0], 'date'] = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

In [8]:
# to make sure it's corrected
training_info['date'] = pd.to_datetime(training_info['date'], format='%Y-%m-%d %H:%M:%S')
training_info.dtypes

mid                    int64
date          datetime64[ns]
body                  object
recipients            object
dtype: object

## 2.2 Merge dataframes

In [9]:
# training
training['mid'] = training['mids'].apply(lambda x: x.split(' '))
del training['mids']
training = training.explode('mid').astype({'mid': 'int64'})
training.reset_index(drop=True, inplace=True)
print("training: ", training.shape)

# test
test['mid'] = test['mids'].apply(lambda x: x.split(' '))
del test['mids']
test = test.explode('mid').astype({'mid': 'int64'})
test.reset_index(drop=True, inplace=True)
print("test: ", test.shape)

training.head()

training:  (43613, 2)
test:  (2362, 2)


,sender,mid
0,karen.buckley@enron.com,158713
1,karen.buckley@enron.com,158697
2,karen.buckley@enron.com,200301
3,karen.buckley@enron.com,158679
4,karen.buckley@enron.com,278595


In [10]:
df_train = pd.merge(training, training_info, how='left', on='mid')
df_test = pd.merge(test, test_info, how='left', on='mid')
df_train.head()

,sender,mid,date,body,recipients
0,karen.buckley@enron.com,158713,2001-03-30 07:07:00,Hi JasonTed let me know of the situation with ...,jason.wolfe@enron.com
1,karen.buckley@enron.com,158697,2001-04-10 02:41:00,"All,John Lavorato has requested your attendanc...",elizabeth.shim@enron.com russell.ballato@enron...
2,karen.buckley@enron.com,200301,2001-04-10 12:41:00,"All,John Lavorato has requested your attendanc...",elizabeth.shim@enron.com russell.ballato@enron...
3,karen.buckley@enron.com,158679,2001-04-18 02:02:00,AllA reminder that Philip Allen will be presen...,vladi.pimenov@enron.com russell.ballato@enron....
4,karen.buckley@enron.com,278595,2001-04-19 21:31:00,"All,We will be scheduling the next recruitment...",thomas.a.martin@enron.com ed.mcmichael@enron.c...


In [12]:
#sort by date
df_train= df_train.sort_values(by='date').reset_index(drop=True)
print(df_train.shape)
print("Unique senders: ", df_train['sender'].nunique())
df_train.head()

(43613, 5)
Unique senders:  125


,sender,mid,date,body,recipients
0,michelle.cash@enron.com,9716,1998-12-21 05:29:00,"Brent,Attached is a form indemnification agree...",brent.hendry@enron.com mark.e.taylor@enron.com
1,christian.yoder@enron.com,7830,1999-03-02 07:30:00,"As you are already aware, the West desk is ma...",elizabeth.sager@enron.com mark.e.taylor@enron....
2,larry.f.campbell@enron.com,90523,1999-05-03 09:44:00,Go for it!George Robinson04/27/99 04:14 PMTo: ...,butch.russell@enron.com george.robinson@enron.com
3,larry.f.campbell@enron.com,89475,1999-05-05 05:27:00,Just a short message to apprise everyone of th...,william.kendrick@enron.com rick.cates@enron.co...
4,larry.f.campbell@enron.com,89618,1999-05-05 05:27:00,Just a short message to apprise everyone of th...,william.kendrick@enron.com rick.cates@enron.co...


## 2.4 Construct address book set
AB[sender][recipients] = number of emails<br>
for each Enron user u, we build an address book set AB(u), which is the list of all recipient addresses in the messages sent by user u. 


In [15]:
def email_counts_by_user(same_user_df):
    # 'sum' here is adding two lists into one big list
    all_recipients = same_user_df['recipients'].str.split(' ').sum()
    # remove address with bad format
    return pd.Series(Counter(recipient for recipient in all_recipients if '@' in recipient ))

In [16]:
email_counts = df_train.groupby("sender").apply(email_counts_by_user).reset_index()
email_counts.rename(columns = {'level_1':'recipient', 0:'nb_emails_s2r'}, inplace=True)
email_counts

,sender,recipient,nb_emails_s2r
0,alan.aronowitz@enron.com,anita.fam@enron.com,2
1,alan.aronowitz@enron.com,william.bradford@enron.com,3
2,alan.aronowitz@enron.com,sara.shackleton@enron.com,40
3,alan.aronowitz@enron.com,mark.e.taylor@enron.com,81
4,alan.aronowitz@enron.com,janice.moore@enron.com,1
...,...,...,...
23129,w..cantrell@enron.com,sara.solorio@enron.com,1
23130,w..cantrell@enron.com,bilal.bajwa@enron.com,1
23131,w..cantrell@enron.com,susan.w.pereira@enron.com,3
23132,w..cantrell@enron.com,thomas.a.martin@enron.com,1


In [17]:
# write to dictionary
AB = {}
sender_list =  email_counts['sender'].unique()
for s in sender_list:
    AB[s] = {}
for index, row in email_counts.iterrows():  
    r = row['recipient']
    if '@' in r: 
        AB[row['sender']][r] = row['nb_emails_s2r']

In [18]:
print(AB['alex@pira.com'])
# print(AB['karen.buckley@enron.com'])
# print(AB['karen.buckley@enron.com']['aamir.maniar@enron.com'])

{'alex@pira.com': 77}


# 3. Baselines: Using Textual Content

First of all, we represent textual contents in tf-idf and normalized the vectors to length 1.0

In [19]:
vectorizer = TfidfVectorizer(stop_words = 'english', lowercase=True)
vectorizer.fit(np.concatenate((df_train.body.values,df_test.body.values)))

tfidf = vectorizer.transform(df_train.body.values)
print(tfidf.shape)

tfidf_test = vectorizer.transform(df_test.body.values) 
print(tfidf_test.shape)

(43613, 194081)
(2362, 194081)


## 3.1 TfIdf-Centroid

Build for each recipient _ri_ in the Address Book AB(u), a TF-IDF centroid vector _Centroid(ri)_ by summing up the normalized TF-IDF vectors of all messages that were sent from _u_ to _ri_.


When testing, for each test message we computed the cosine similarities between the message’s TF-IDF vector representation and the |AB(u)| Centroid vectors. The |AB(u)| recipients are then ranked according to these cosine similarity scores.

In [20]:
%%time 

sender_list =  email_counts['sender'].unique()
centroid = {}

for u in sender_list:
    centroid[u]={}

for index, row in df_train.iterrows():
    u = row['sender']
    for ri in row['recipients'].split(' '):
        # double check if ri is in Address Book of u
        if ri in AB[u].keys(): 
            if ri in centroid[u].keys():
                centroid[u][ri]+=tfidf[index]
            else:
                centroid[u][ri]=tfidf[index]

CPU times: user 2min, sys: 892 ms, total: 2min 1s
Wall time: 2min 1s


In [21]:
centroid['alan.aronowitz@enron.com']['anita.fam@enron.com']

<1x194081 sparse matrix of type '<class 'numpy.float64'>'
	with 323 stored elements in Compressed Sparse Row format>

In [22]:
centroid['alex@pira.com']

{'alex@pira.com': <1x194081 sparse matrix of type '<class 'numpy.float64'>'
 	with 97 stored elements in Compressed Sparse Row format>}

In [23]:
# Write to disk since the computation time is quite long
with open('centroid_tfidf.pkl', 'wb') as f:
    pickle.dump(centroid, f, pickle.HIGHEST_PROTOCOL)  

Compute cosine similarity

In [24]:
prediction_tfidf_centroid = {}

for index, row in df_test.iterrows():
    # iterate over each email in test set
    u = row['sender']
    vec = tfidf_test[index]
    similarity = []
    for ri in centroid[u].keys():
        similarity.extend(cosine_similarity(centroid[u][ri],vec).reshape(1))
    # rank according to the cosine similarity scores and take top10
    argmax = np.array(similarity).argsort()[::-1][:10]
    # get predictions
    prediction_tfidf_centroid[row['mid']] = [list(centroid[u].keys())[arg] for arg in argmax]

Save predictions 

In [25]:
with open('prediction_tfidf_centroid.txt', 'w') as f:
    f.write('mid,recipients' + '\n')
    for index,row in df_test.iterrows():
        mid = row.mid
        pred = prediction_tfidf_centroid[mid]
        f.write(str(mid) + ',' + ' '.join(pred) + '\n')

## 3.2 K-Nearest-Neighbors (Knn-30)

Given a message _mu_ from user _u_ addressed to a set of recipients R(mu) = {r1,..,r|R(mu)|}, we found its 30 most similar messages in the training set. The notion of similarity here is also defined as the cosine distance between the text of two normalized TF-IDF vectors. 

With the top 30 most similar messages selected from the training set, we then computed the weight of each recipient _ri_ in |AB(u)| according to the sum of similarity scores of the messages (from the top 30 messages) in which _ri_ was one of the recipients. All recipients are ranked according to this final weight. We refer to this method as Knn-30.

In [26]:
n_neighbor=30
prediction_knn30 = {}

for index, row in df_test.iterrows():
    # iterate over each email in test set
    u = row['sender']
    # rank according to the cosine similarity scores 
    similarity = cosine_similarity(tfidf,tfidf_test[index]).reshape(tfidf.shape[0])
    # take top 30 most similar messages
    i_argmax = np.array(similarity).argsort()[::-1][:n_neighbor]
   
    # computed the weight of each recipient
    score = {}
    # from the top 30 messages
    for i in i_argmax:
        # in which ri was one of the recipients
        for ri in df_train.loc[i]['recipients'].split(' '):
            # Double check 
            if ri in AB[u].keys():
                if ri in score.keys():
                    score[ri]+=similarity[i]
                else:
                    score[ri]=similarity[i]
    # Write predictions
    prediction_knn30[row['mid']] = sorted(score.keys(),reverse=True,key=lambda x : score[x])[:10]

Save predictions

In [27]:
with open('prediction_knn30.txt', 'w') as f:
    f.write('mid,recipients' + '\n')
    for index,row in df_test.iterrows():
        mid = row.mid
        pred = prediction_knn30[mid]
        f.write(str(mid) + ',' + ' '.join(pred) + '\n')

# 4. Reranking Using Network Information

So far we have considered only the textual contents of emails in both recipient prediction tasks. Yet, it is reasonable to consider other features for these problems, such as the number of received messages, number of sent messages, number of times two recipients were copied in the same message, etc. 

## 4.1 Crossvalidation 
Calculate the textual similarity scores using a cross-validation procedure in the training set.
- Using a 10-fold cross-validation procedure: we compute the score for the knn-30 on 90% of the training data and use it to make predictions in the remaining 10%. 
- Eventually, each training set examples will have, associated with it, a list of email addresses (from the top 30 messages selected by Knn-30) and their predicted scores. 
- Now we have, for each message _mu_ in user u’s training set, a score _KnnScore(mu,ri)_ associated with each recipient _ri_ ∈ AB(u). 

In [28]:
from sklearn.model_selection import KFold, LeaveOneOut
import json

#### Get df_train

In [29]:
def knn_score(train_index, test_index, n_neighbor):
    X_train, X_test = df_train.loc[train_index], df_train.loc[test_index]
    pred_knn30_score = {}
    
    for index, row in X_test.iterrows():
        u = row['sender']
        similarity = cosine_similarity(tfidf[train_index],tfidf[index]).reshape(len(train_index))
        i_argmax = np.array(similarity).argsort()[::-1][:n_neighbor]
        
        score = {}
        for i in i_argmax:
            # in which ri was one of the recipients
            for ri in df_train.loc[train_index[i]]['recipients'].split(' '):
                # Double check 
                if ri in AB[u].keys():
                    if ri in score.keys():
                        score[ri]+=similarity[i]
                    else:
                        score[ri]=similarity[i]
        pred_knn30_score[index] = score
    return pred_knn30_score

In [ ]:
# V1 : use all mails in df_train

# n_neighbor=30
# kf = KFold(n_splits=10)
# knn_scores = {}
# for train_index, test_index in kf.split(df_train):
#     #print("*******")
#     print(test_index)
#     scores = knn_score(train_index, test_index , n_neighbor)
#     #print(scores.keys())
#     knn_scores.update(scores)
#     #print(knn_scores.keys())
# 
# with open('knn_scores.json', 'w') as f:
#     json.dump(knn_scores, f)

# ##############################################
# V2: add df_train[df_train.sender==s]

# n_neighbor=30
# knn_scores_v2 = {}
# for u in AB.keys():
#     train_ = df_train[df_train['sender']==u]
#     all_index = train_.index
#     kf = KFold(n_splits=10)
#     print("*******")
#     print(u)
#     # kFold iterator yields the positional indices of train_ 
#     for tr_idx, te_idx in kf.split(train_):
#         train_index = all_index[tr_idx]
#         test_index = all_index[te_idx]
#         scores = knn_score(train_index, test_index , n_neighbor)
#         print(scores.keys())
#         knn_scores_v2.update(scores)
#
# assert len(knn_scores_v2)==len(df_train)
# 
# with open('knn_scores_v2.json', 'w') as f:
#     json.dump(knn_scores_v2, f)

# ##############################################
# V3: Leave-one-out
# add df_train[df_train.sender==s]

# n_neighbor=30
# knn_scores_v3 = {}
# for u in AB.keys():
#     train_ = df_train[df_train['sender']==u]
#     all_index = train_.index
#     loo = LeaveOneOut()
#     print("*******")
#     print(u) 
#     for tr_idx, te_idx in loo.split(train_):
#         train_index = all_index[tr_idx]
#         test_index = all_index[te_idx]
#         scores = knn_score(train_index, test_index , n_neighbor)
#         # print(scores.keys())
#         knn_scores_v3.update(scores)
# 
# assert len(knn_scores_v3)==len(df_train)        
# 
# with open('knn_scores_v3.json', 'w') as f:
#     json.dump(knn_scores_v3, f)

In [118]:
# V4: one-versus-all KNN
# add df_train[df_train.sender==s]

n_neighbor=30
knn_scores_v4 = {}
for u in AB.keys():
    train_ = df_train[df_train['sender']==u]
    all_index = train_.index
    print("*******")
    print(u) 
    for idx in all_index:
        similarity = cosine_similarity(tfidf[all_index],tfidf[idx]).reshape(len(train_))
        i_argmax = np.array(similarity).argsort()[::-1][:n_neighbor]
        score = {}
        for i in i_argmax:
            # in which ri was one of the recipients
            for ri in df_train.loc[all_index[i]]['recipients'].split(' '):
                # Double check 
                if ri in AB[u].keys():
                    if ri in score.keys():
                        score[ri]+=similarity[i]
                    else:
                        score[ri]=similarity[i]
        knn_scores_v4[idx] = score

with open('knn_scores_v4.json', 'w') as f:
    json.dump(knn_scores_v4, f)

*******
alan.aronowitz@enron.com
*******
alan.comnes@enron.com
*******
alex@pira.com
*******
amr.ibrahim@enron.com
*******
amy.fitzpatrick@enron.com
*******
andrea.ring@enron.com
*******
andrew.edison@enron.com
*******
andy.zipper@enron.com
*******
barry.tycholiz@enron.com
*******
becky.spencer@enron.com
*******
ben.jacoby@enron.com
*******
beth.cherry@enform.com
*******
bob.shults@enron.com
*******
brad.mckay@enron.com
*******
brian.redmond@enron.com
*******
britt.davis@enron.com
*******
c..giron@enron.com
*******
c..williams@enron.com
*******
cheryl.johnson@enron.com
*******
chris.dorland@enron.com
*******
chris.germany@enron.com
*******
christian.yoder@enron.com
*******
christina.valdez@enron.com
*******
cindy.stark@enron.com
*******
darrell.schoolcraft@enron.com
*******
david.forster@enron.com
*******
david.port@enron.com
*******
david.portz@enron.com
*******
dutch.quigley@enron.com
*******
enron_update@concureworkplace.com
*******
eric.bass@enron.com
*******
errol.mclaughlin@enron

#### Get df_test

In [ ]:
# V1: use all mails in df_train

# n_neighbor=30
# knn_scores_test = {}
# 
# for index, row in df_test.iterrows():
#     # iterate over each email in test set
#     u = row['sender']
#     # rank according to the cosine similarity scores 
#     similarity = cosine_similarity(tfidf,tfidf_test[index]).reshape(tfidf.shape[0])
#     # take top 30 most similar messages
#     i_argmax = np.array(similarity).argsort()[::-1][:n_neighbor]
#    
#     # computed the weight of each recipient
#     score = {}
#     # from the top 30 messages
#     for i in i_argmax:
#         # in which ri was one of the recipients
#         for ri in df_train.loc[i]['recipients'].split(' '):
#             # Double check 
#             if ri in AB[u].keys():
#                 if ri in score.keys():
#                     score[ri]+=similarity[i]
#                 else:
#                     score[ri]=similarity[i]
#     # Write predictions
#     knn_scores_test[index] = score
#
#
# with open('knn_scores_test.json', 'w') as f:
#     json.dump(knn_scores_test, f)

In [33]:
# V2 
# add df_train[df_train.sender==s]

n_neighbor=30
knn_scores_test_v2 = {}

for index, row in df_test.iterrows():
    # iterate over each email in test set
    u = row['sender']
    # take tfidf where sender = u
    all_index = df_train[df_train['sender']==u].index
    tfidf_u = tfidf[all_index]
    print("****")
    print(u)
    print("*")
    # rank according to the cosine similarity scores 
    similarity = cosine_similarity(tfidf_u,tfidf_test[index]).reshape(tfidf_u.shape[0])
    # take top 30 most similar messages
    i_argmax = np.array(similarity).argsort()[::-1][:n_neighbor]
   
    # computed the weight of each recipient
    score = {}
    # from the top 30 messages
    for i in i_argmax:
        print(df_train.loc[all_index[i]]['sender'])
        # in which ri was one of the recipients
        for ri in df_train.loc[all_index[i]]['recipients'].split(' '):
            # Double check 
            if ri in AB[u].keys():
                if ri in score.keys():
                    score[ri]+=similarity[i]
                else:
                    score[ri]=similarity[i]
    # Write predictions
    knn_scores_test_v2[index] = score
    
with open('knn_scores_test_v2.json', 'w') as f:
    json.dump(knn_scores_test_v2, f)

****
karen.buckley@enron.com
*
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
****
karen.buckley@enron.com
*
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
ka

karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
****
karen.buckley@enron.com
*
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
karen.buckley@enron.com
****
kare

amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
****
amr.ibrahim@enron.com
*
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
amr.ibrahim@enron.com
***

****
sylvia.hu@enron.com
*
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
****
sylvia.hu@enron.com
*
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia.hu@enron.com
sylvia

phillip.platter@enron.com
****
phillip.platter@enron.com
*
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
****
phillip.platter@enron.com
*
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.com
phillip.platter@enron.co

...

****
barry.tycholiz@enron.com
*
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
****
barry.tycholiz@enron.com
*
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycholiz@enron.com
barry.tycho

amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
****
amy.fitzpatrick@enron.com
*
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.fitzpatrick@enron.com
amy.f

c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
****
c..williams@enron.com
*
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..

c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
****
c..williams@enron.com
*
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..williams@enron.com
c..

enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_update@concureworkplace.com
****
enron_update@concureworkplace.com
*
enron_update@concureworkplace.com
enron_update@concureworkplace.com
enron_u

## 4.2 Network-Based Features

### 4.2.1 Frequency features

The first set is based on the relative frequency of a recipient’s email address in the training
set. For each recipient we extracted three features: 
- the normalized sent frequency (i.e., the number of messages sent to this recipient divided by the total number of messages sent to all other recipients in this user u’s training set)
- the normalized received frequency (i.e., the number of messages received from this sender divided by the total number of messages received from all other users) 
- normalized sum of the previous two. 

In [35]:
email_counts.head()

,sender,recipient,nb_emails_s2r
0,alan.aronowitz@enron.com,anita.fam@enron.com,2
1,alan.aronowitz@enron.com,william.bradford@enron.com,3
2,alan.aronowitz@enron.com,sara.shackleton@enron.com,40
3,alan.aronowitz@enron.com,mark.e.taylor@enron.com,81
4,alan.aronowitz@enron.com,janice.moore@enron.com,1


In [36]:
#normalized sent frequency
email_counts['nb_emails_s'] = email_counts.groupby('sender')['nb_emails_s2r'].transform('sum')
email_counts['sent_frequency'] = email_counts['nb_emails_s2r']/email_counts['nb_emails_s']

In [37]:
#normalized recienved frequency 
email_counts['nb_emails_2r'] = email_counts.groupby('recipient')['nb_emails_s2r'].transform('sum')
email_counts['received_frequency'] = email_counts['nb_emails_s2r']/email_counts['nb_emails_2r']

In [38]:
email_counts.head()

,sender,recipient,nb_emails_s2r,nb_emails_s,sent_frequency,nb_emails_2r,received_frequency
0,alan.aronowitz@enron.com,anita.fam@enron.com,2,676,0.002959,5,0.400000
1,alan.aronowitz@enron.com,william.bradford@enron.com,3,676,0.004438,409,0.007335
2,alan.aronowitz@enron.com,sara.shackleton@enron.com,40,676,0.059172,1428,0.028011
3,alan.aronowitz@enron.com,mark.e.taylor@enron.com,81,676,0.119822,1517,0.053395
4,alan.aronowitz@enron.com,janice.moore@enron.com,1,676,0.001479,45,0.022222


In [39]:
print(email_counts[email_counts['sender']== 'alex@pira.com']['sent_frequency'] )
print(email_counts[email_counts['recipient']== 'alex@pira.com']['received_frequency'])

537    1.0
Name: sent_frequency, dtype: float64
537     0.987179
6665    0.012821
Name: received_frequency, dtype: float64


In [40]:
# write to dictionary
sent_frequency = {}
received_frequency = {}

sender_list =  email_counts['sender'].unique()
for s in sender_list:
    sent_frequency[s] = {}

recipient_list =  email_counts['recipient'].unique()
for r in recipient_list:
    received_frequency[r] = {}

for index, row in email_counts.iterrows():    
    sent_frequency[row['sender']][row['recipient']] = row['sent_frequency']
    received_frequency[row['recipient']][row['sender']] = row['received_frequency']

In [41]:
print(sent_frequency['alex@pira.com'])
print(received_frequency['alex@pira.com'])

{'alex@pira.com': 1.0}
{'alex@pira.com': 0.9871794871794872, 'holden.salisbury@enron.com': 0.01282051282051282}


In [ ]:
# normalized sum of the previous two

In [42]:
with open('sent_frequency.json', 'w') as f1:
    json.dump(sent_frequency, f1)
    
with open('received_frequency.json', 'w') as f2:
    json.dump(received_frequency, f2)

### 4.2.2 Recency features

The second set of network-based features uses the information in the latest messages sent by the user. Similar to the Frequency features above, we extract the normalized sent frequency of all users in the training set. But instead of using the entire training set for the extraction, we only use the last 20, last 50 or last 100 messages(dimension of recency).

In [43]:
print(min(df_train['date']), max(df_train['date']))
print(min(df_test['date']), max(df_test['date']))

1998-12-21 05:29:00 2002-03-08 14:09:25
2001-11-02 05:25:29 2002-06-24 13:15:28


In [44]:
def getRecencyFeatures(sender, dim_recency, max_date):
    # select top dim_recency
    tmp = df_train[(df_train['sender']==sender)&(df_train['date']<=max_date)].sort_values(by='date',ascending=False)[:dim_recency] 
    all_recipients = tmp['recipients'].str.split(' ').sum()
    # number of emails sent from sender to each recipient
    counter = Counter(recipients for recipients in all_recipients)
    # total number of emails sent from sender
    total = sum(counter.values())
    # compute sent frequency
    recency_features = {r: c/total for r, c in counter.items()} 
    return recency_features

In [45]:
getRecencyFeatures('alex@pira.com', 20, '2001-04-25 15:15:00')

{'alex@pira.com': 1.0}

## 4.3 Reranking
For each training message $m_s$ with $J$ recipients , we
created $|AB(s)|$ new binary examples: $J$ positive examples associated with the true recipients and $|AB(s)| − J$ negative examples with the features associated with all other non-recipients addresses in the Address Book $AB(s)$.<br>
In practice, instead of using all $|AB(u)| − J$ negative examples, we used only a random selection of 20% of them. This
provided an effective speed up in training time and did not
seem to affect performance.
Therefore, each message $m_s$ originated $J+0.2(|AB(u)|-J)$ binary examples;
and each binary example is related to an email address $r_i$
from the address book. These binary example contains the
following features: $KnnScore(m_s , r_i )$, Frequency
features of $r_i$ , Recency features of $r_i$

In [147]:
# knn_scores = json.load(open("knn_scores.json","r"))
knn_scores_v2 = json.load(open("knn_scores_v2.json","r"))
# knn_scores_v3 = json.load(open("knn_scores_v3.json","r"))
# knn_scores_v4 = json.load(open("knn_scores_v4.json","r"))


# knn_scores_test = json.load(open("knn_scores_test.json","r"))
knn_scores_test_v2 = json.load(open("knn_scores_test_v2.json","r"))

In [46]:
def header_address_ressemblance(text, address):
    head = text[:20].lower()
    name = address[:address.index('@')].split('.')
    for n in name:
        if len(n)>2:
            if n in head:
                return True
    return False

In [155]:
def transform_train(row_index):
    u = df_train.loc[row_index]['sender']
    max_date = df_train.loc[row_index]['date']
    true_recipients = df_train.loc[row_index]['recipients'].split(' ')
    header = df_train.loc[row_index]['body'][:20]
    
    recency_features = getRecencyFeatures(u, 20, max_date)
    #knn = knn_scores[row_index]
    knn = knn_scores_v2[str(row_index)]
    
    features =  np.zeros((len(knn),5))
    labels = np.zeros((len(knn),1))
    i=0
    #print("**")
    #print("**recency_features")
    #print(recency_features)
    #print("**knn")
    #print(len(knn))
    #print(knn)
    
    #for r in AB[u].keys():
    for r in knn.keys():
        # features
        rec, r_freq = 0., 0.
        knn_s = knn[r]
        s_freq = sent_frequency[u][r]
        if u in received_frequency.keys():
            r_freq = received_frequency[u].get(r,0)
        if r in recency_features.keys():
            rec = recency_features[r]
        head = 1.0 * header_address_ressemblance(header,r)
        features[i] = [knn_s,s_freq,r_freq,rec,head]
        # label
        if r in true_recipients:
            labels[i]=1
        i+=1
        #print("# recipient:", r)
        #print("received_frequency:", r_freq)
        #print("sent_frequency:",s_freq )
    
    return features, labels


def transform_test(row_index):
    u = df_test.loc[row_index]['sender']
    max_date = df_test.loc[row_index]['date']
    header = df_test.loc[row_index]['body'][:20]
    
    recency_features = getRecencyFeatures(u, 20, max_date)
    #knn = knn_scores_test_v2[row_index]
    knn = knn_scores_test_v2[str(row_index)]
    
    features =  np.zeros((len(knn),5))
    i=0
    #for r in AB[u].keys():
    for r in knn.keys():
        # features
        rec, r_freq = 0., 0.
        knn_s = knn[r]
        s_freq = sent_frequency[u][r]
        if u in received_frequency.keys():
            r_freq = received_frequency[u].get(r,0)
        if r in recency_features.keys():
            rec = recency_features[r]
        head = 1.0 * header_address_ressemblance(header,r)
        features[i] = [knn_s,s_freq,r_freq,rec,head]
        i+=1
    return features, knn

Build training features and labels

In [154]:
X_tr, y_tr = [], []
for idx in df_train.index.values:
    if idx%1000==0:
        print(idx)
    features,labels = transform_train(idx)
    X_tr.extend(features)
    y_tr.extend(labels)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000


Train a logistic model on training data

In [156]:
y_tr = np.array(y_tr).ravel()

clf = LogisticRegression(C=0.001)
clf.fit(X_tr, y_tr)

/Users/zhengjie/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

Build test features and make predictions

In [157]:
with open('prediction_reranking_v2.txt', 'w') as f:
    f.write('mid,recipients' + '\n')
    
    # make predictions
    for idx in df_test.index.values:
        if idx%1000==0:
            print(idx)
        features, knn = transform_test(idx)
        u = df_test.loc[idx]['sender']
        order = clf.predict_proba(features)[:,1].argsort()[::-1]
        order = order[:10]
        mid = df_test.loc[idx]['mid']
        pred = [list(knn.keys())[i] for i in order]
        
        f.write(str(mid) + ',' + ' '.join(pred) + '\n')

0
1000
2000
